In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import *
import random
import matplotlib.pyplot as plt


In [2]:
print(os.getcwd())

/root/TSTF/torch


In [4]:
data = ['coin']
model_name = 'trTFMLP'
num = 100    # Model k per loss

for d in range(len(data)):
    #############################################################################################
    target_X= pd.read_csv(f"../data/{data[d]}/train_input_7.csv").iloc[:,1:].values.astype(np.float32)
    target_y =pd.read_csv(f"../data/{data[d]}/train_output_7.csv").iloc[:,1:].values.astype(np.float32)
    
    
    target_X_val= target_X[-round(target_X.shape[0]*0.2):,:].astype(np.float32)
    target_y_val =target_y[-round(target_y.shape[0]*0.2):].astype(np.float32)
    
    
    target_X = target_X[:-round(target_X.shape[0]*0.2),:].astype(np.float32)
    target_y = target_y[:-round(target_y.shape[0]*0.2)].astype(np.float32)
    test_X= pd.read_csv(f"../data/{data[d]}/val_input_7.csv").iloc[:,1:].values.astype(np.float32)
    test_y =pd.read_csv(f"../data/{data[d]}/val_output_7.csv").iloc[:,1:].values.astype(np.float32)
    
    
    #############################################################################################
    
    folder_path = 'resulttf/test/'
    folder_path2 = 'resulttf/val/'
    
    all_files = os.listdir(folder_path)
    all_files2 = os.listdir(folder_path2)
    print(f"--- 파일 검색 디버깅 ---")
    print(f"검색 중인 폴더: {folder_path}")
    print(f"폴더 안의 모든 파일: {all_files}")
    search_prefix = f"{model_name}_{data[d]}"
    print(f"찾고 있는 파일 시작 부분(prefix): '{search_prefix}'")
    print("--- 디버깅 끝 ---")
    files = sorted([f for f in all_files if f.startswith(f"{model_name}_{data[d]}") and f.endswith("pred.csv")])
    dataframes = [pd.read_csv(os.path.join(folder_path, f)) for f in files]
    
    files2 = sorted([f for f in all_files2 if f.startswith(f"{model_name}_{data[d]}") and f.endswith("pred.csv")])
    dataframes2 = [pd.read_csv(os.path.join(folder_path2, f)) for f in files2]
    
    all=np.array([np.stack(dataframes[i].iloc[:,1:].values.reshape(num,-1, target_y.shape[1])) for i in range(len(dataframes))])#.reshape(num*5,-1,24)
    all_val=np.array([np.stack(dataframes2[i].iloc[:,1:].values.reshape(num,-1, target_y.shape[1])) for i in range(len(dataframes2))])#.reshape(num*5,-1,24)
    #############################################################################################
    all_ens_lst, all_ens_rmse_lst = [], []
    bolt_lst, bolt_rmse_lst        = [], []
    
    inv_lst, inv_rmse_lst          = [] , []        ### NEW: 역수 가중치 결과 저장용
    mean_rmse_lst                  = []             ### NEW: mean 평가 지표 기록용

    # 251015 ===== [ADD-INIT] 디버깅 수집 컨테이너 & 경로 =====
    # 1) 손실별 test RMSE 수집
    per_loss_test_rmse = {'MAE': [], 'MAPE': [], 'MASE': [], 'MSE': [], 'SMAPE': []}
    
    # 2) β 그리드 및 β별 가중치 저장 (각 부트스트랩에서의 weight 벡터 5개)
    beta_grid = np.arange(1, 10.2, 0.2)  # 1.0 ~ 10.0 (step 0.2)
    weights_by_beta = {float(np.round(b, 3)): [] for b in beta_grid}  # {beta: [w(5,)]}
    
    # 3) 각 부트스트랩에서 "최적 β"와 그때의 가중치/성능 기록
    best_beta_rows = []  # dict(iter, beta, bolt_rmse, w_mae, w_mape, w_mase, w_mse, w_smape)
    
    # 4) 저장 경로 보장
    os.makedirs('inference', exist_ok=True)

    SAVE_BREAKDOWN_PER_ITER = False   # per-iter 저장 끄기(용량 절감)
    samples = []                      # 각 iter에서 뽑은 모델 인덱스 저장
    
    p = 10
    b = 50
    np.random.seed(100)       ### NEW: mean 평가 지표 기록용

    
    for i in range(b):
        # --------------------------------------------------------------
        # 1) 무작위 인덱스 샘플링
        #matrix = np.arange(100).reshape(10, 10)
        nums1 = random.sample(range(num), k=p)
        nums2 = random.sample(range(num), k=p)
        nums3 = random.sample(range(num), k=p)
        nums4 = random.sample(range(num), k=p)
        nums5 = random.sample(range(num), k=p)

        # ▼ 추가: 나중에 재현하려고 저장
        samples.append((nums1, nums2, nums3, nums4, nums5))

        #nums1 = matrix[i]
        #nums2 = matrix[i]
        #nums3 = matrix[i]
        #nums4 = matrix[i]
        #nums5 = matrix[i]
        # --------------------------------------------------------------
        # 2) all_Ens  (median 앙상블은 그대로)
        score      = np.concatenate([all[0][nums1], all[1][nums2],
                                       all[2][nums3], all[3][nums4],
                                       all[4][nums5]], axis=0)
        
        all_ens    = np.median(score, axis=0).flatten()
        all_ens_rmse = np.sqrt(mean_squared_error(test_y.flatten(), all_ens))
        all_ens_lst.append(all_ens)
        all_ens_rmse_lst.append(all_ens_rmse)

        
        # --- [순서 변경] Breakdown 생성을 위해 performance와 gd를 먼저 계산 ---
        mae_val = np.median(all_val[0][nums1],axis=0).flatten()
        mape_val = np.median(all_val[1][nums2],axis=0).flatten()
        mase_val = np.median(all_val[2][nums3],axis=0).flatten()
        mse_val = np.median(all_val[3][nums4],axis=0).flatten()
        smape_val = np.median(all_val[4][nums5],axis=0).flatten()
        
        rmse_mae_val = np.sqrt(mean_squared_error(target_y_val.flatten(),mae_val))
        rmse_mape_val = np.sqrt(mean_squared_error(target_y_val.flatten(),mape_val))
        rmse_mase_val = np.sqrt(mean_squared_error(target_y_val.flatten(),mase_val))
        rmse_mse_val = np.sqrt(mean_squared_error(target_y_val.flatten(),mse_val))
        rmse_smape_val = np.sqrt(mean_squared_error(target_y_val.flatten(),smape_val))

        performance = np.array([rmse_mae_val, rmse_mape_val,rmse_mase_val,rmse_mse_val,rmse_smape_val])

        fin_pred_mae = np.median(all[0][nums1], axis=0).flatten()
        fin_pred_mape = np.median(all[1][nums2], axis=0).flatten()
        fin_pred_mase = np.median(all[2][nums3], axis=0).flatten()
        fin_pred_mse = np.median(all[3][nums4], axis=0).flatten()
        fin_pred_smape = np.median(all[4][nums5], axis=0).flatten()
        gd = np.stack([fin_pred_mae, fin_pred_mape, fin_pred_mase, fin_pred_mse, fin_pred_smape], axis=0)

        # (참고용) 손실별 test RMSE 수집
        rmse_mae_test    = np.sqrt(mean_squared_error(test_y.flatten(),  fin_pred_mae))
        rmse_mape_test   = np.sqrt(mean_squared_error(test_y.flatten(),  fin_pred_mape))
        rmse_mase_test   = np.sqrt(mean_squared_error(test_y.flatten(),  fin_pred_mase))
        rmse_mse_test    = np.sqrt(mean_squared_error(test_y.flatten(),  fin_pred_mse))
        rmse_smape_test  = np.sqrt(mean_squared_error(test_y.flatten(),  fin_pred_smape))
        per_loss_test_rmse['MAE'].append(rmse_mae_test)
        per_loss_test_rmse['MAPE'].append(rmse_mape_test)
        per_loss_test_rmse['MASE'].append(rmse_mase_test)
        per_loss_test_rmse['MSE'].append(rmse_mse_test)
        per_loss_test_rmse['SMAPE'].append(rmse_smape_test)


        # --- 이제 상세 Breakdown 생성 코드를 실행 ---
        if i == 0:
            y_true_flat = test_y.flatten()
            num_timesteps = all_ens.shape[0]
            loss_names = ['mae', 'mape', 'mase', 'mse', 'smape']
            
            # (1) Median 앙상블 Breakdown
            iter0_median_rmse = np.sqrt(mean_squared_error(y_true_flat, all_ens))
            print(f"\nINFO (iter 0): Median Ensemble RMSE = {iter0_median_rmse:.5f}")
            median_rows = []
            for t in range(num_timesteps):
                row = {'iter': i, 't': t, 'y_true': y_true_flat[t], 'median_pred': all_ens[t], 'squared_error': (y_true_flat[t] - all_ens[t])**2, 'overall_rmse': iter0_median_rmse}
                for loss_idx, name in enumerate(loss_names):
                    row[f'base_pred_{name}'] = gd[loss_idx, t]
                median_rows.append(row)
            pd.DataFrame(median_rows).to_csv(f'inference/{model_name}_{data[d]}_median_breakdown_rmse.csv', index=False)
            print(f"INFO: Median breakdown for iter 0 saved.")

            # (2) 특정 Beta(1, 3, 5) 값에 대한 BOLT 앙상블 Breakdown
            for target_beta in [1, 3, 5]:
                weights = np.exp(-target_beta * performance)
                weights /= np.sum(weights)
                ensemble_prediction = np.dot(weights, gd)
                iter0_bolt_rmse = np.sqrt(mean_squared_error(y_true_flat, ensemble_prediction))
                print(f"INFO (iter 0): BOLT (beta={target_beta}) RMSE = {iter0_bolt_rmse:.5f}")
                bolt_rows = []
                for t in range(num_timesteps):
                    row = {'iter': i, 'beta': target_beta, 't': t, 'y_true': y_true_flat[t], 'ensemble_pred': ensemble_prediction[t], 'squared_error': (y_true_flat[t] - ensemble_prediction[t])**2, 'overall_rmse': iter0_bolt_rmse}
                    for loss_idx, name in enumerate(loss_names):
                        row[f'base_pred_{name}'] = gd[loss_idx, t]
                        row[f'w_{name}'] = weights[loss_idx]
                    bolt_rows.append(row)
                pd.DataFrame(bolt_rows).to_csv(f'inference/{model_name}_{data[d]}_bolt_breakdown_beta_{target_beta}_rmse.csv', index=False)
                print(f"INFO: BOLT breakdown for beta={target_beta} saved.")
            print("-" * 20)
        
        # --------------------------------------------------------------
    
        # --------------------------------------------------------------
        # 4) mean 평가지표(다섯 RMSE의 평균) 기록  ### NEW
        mean_rmse   = np.mean(score, axis=0).flatten()
        all_mean_rmse = np.sqrt(mean_squared_error(test_y.flatten(), mean_rmse))
        mean_rmse_lst.append(all_mean_rmse)
    
        bolt_rmse_lst_ = []
        bolt = []

        # 251015 ===== [ADD-BETA-PREP] β별 가중치 수집 준비 =====
        weights_seq = []   # β 순서대로 weight(길이 5) 기록

        for beta in np.arange(1,10.2,0.2):
            #beta = 1 # 조정 파라미터
            weights = np.exp(-beta * performance)
            # gd 계산이 루프 밖으로 이동했으므로 여기서는 재정의할 필요 없음
            
            normalized_weights = weights / np.sum(weights)

            # ===== [ADD-WEIGHTS-CAPTURE] β별 가중치 저장 =====
            weights_seq.append(normalized_weights.copy())
            weights_by_beta[float(np.round(beta, 3))].append(normalized_weights.copy())

            # 각 모델의 예측값에 가중치를 부여하여 앙상블 예측 생성
            ensemble_prediction = np.dot(normalized_weights, gd)
            bolt.append(ensemble_prediction)
            bolt_rmse=np.sqrt(mean_squared_error(test_y.flatten(),ensemble_prediction.flatten()))
            bolt_rmse_lst_.append(bolt_rmse)
            
        
        bolt_lst.append(bolt)
        bolt_rmse_lst.append(bolt_rmse_lst_)
        # 251015 ===== [ADD-BEST-BETA] 이 부트스트랩에서 최적 β 기록 =====
        best_idx = int(np.argmin(bolt_rmse_lst_))
        best_beta_val = float(np.round(beta_grid[best_idx], 3))
        best_weights = weights_seq[best_idx]
        best_bolt_rmse = bolt_rmse_lst_[best_idx]
        best_beta_rows.append({
            'iter': i,
            'beta': best_beta_val,
            'bolt_rmse': best_bolt_rmse,
            'w_mae':   best_weights[0],
            'w_mape':  best_weights[1],
            'w_mase':  best_weights[2],
            'w_mse':   best_weights[3],
            'w_smape': best_weights[4],
        })

        best_ens_pred = bolt[best_idx].flatten()

        show_idx = range(best_ens_pred.shape[0])  # <<<<< 모든 시점

        dbg_rows = []
        y_true_flat = test_y.flatten()  # 대응 GT
        
        for k in show_idx:
            base_preds_k = gd[:, k]  # 길이 5: [MAE, MAPE, MASE, MSE, SMAPE]
            recon = float(np.dot(best_weights, base_preds_k))
            dbg_rows += [
                {'iter': i, 'beta': best_beta_val, 't': k, 'loss': 'MAE',   'base_pred': base_preds_k[0], 'weight': best_weights[0], 'weighted': base_preds_k[0]*best_weights[0],
                 'ensemble': best_ens_pred[k], 'reconstructed': recon, 'y_true': y_true_flat[k]},
                {'iter': i, 'beta': best_beta_val, 't': k, 'loss': 'MAPE',  'base_pred': base_preds_k[1], 'weight': best_weights[1], 'weighted': base_preds_k[1]*best_weights[1],
                 'ensemble': best_ens_pred[k], 'reconstructed': recon, 'y_true': y_true_flat[k]},
                {'iter': i, 'beta': best_beta_val, 't': k, 'loss': 'MASE',  'base_pred': base_preds_k[2], 'weight': best_weights[2], 'weighted': base_preds_k[2]*best_weights[2],
                 'ensemble': best_ens_pred[k], 'reconstructed': recon, 'y_true': y_true_flat[k]},
                {'iter': i, 'beta': best_beta_val, 't': k, 'loss': 'MSE',   'base_pred': base_preds_k[3], 'weight': best_weights[3], 'weighted': base_preds_k[3]*best_weights[3],
                 'ensemble': best_ens_pred[k], 'reconstructed': recon, 'y_true': y_true_flat[k]},
                {'iter': i, 'beta': best_beta_val, 't': k, 'loss': 'SMAPE', 'base_pred': base_preds_k[4], 'weight': best_weights[4], 'weighted': base_preds_k[4]*best_weights[4],
                 'ensemble': best_ens_pred[k], 'reconstructed': recon, 'y_true': y_true_flat[k]},
            ]
        if SAVE_BREAKDOWN_PER_ITER:
            # CSV로 누적 append 저장 (파일 없으면 헤더 포함)
            dbg_path = f'inference/{model_name}_{data[d]}_value_weight_breakdown.csv'
            exists = os.path.exists(dbg_path)
            pd.DataFrame(dbg_rows).to_csv(dbg_path, mode='a', header=not exists, index=False)
            print(
            f"[iter {i}] best β={best_beta_val:.3f} | BOLT_RMSE={best_bolt_rmse:.6f} | "
            f"weights(MAE,MAPE,MASE,MSE,SMAPE)="
            f"({best_weights[0]:.4f}, {best_weights[1]:.4f}, {best_weights[2]:.4f}, {best_weights[3]:.4f}, {best_weights[4]:.4f})"
            )
        # --------------------------------------------------------------
        # 7) **역수 가중치 앙상블** ### NEW
        inv_weights   = 1.0 / performance
        inv_weights  /= np.sum(inv_weights)         # 정규화
    
        ens_pred_inv  = np.dot(inv_weights, gd)
        inv_lst.append(ens_pred_inv)
    
        inv_rmse      = np.sqrt(mean_squared_error(test_y.flatten(),
                                                  ens_pred_inv.flatten()))
        inv_rmse_lst.append(inv_rmse)
    
# ============================ ▼▼▼ [최종] 평균 기반 Breakdown 생성 및 요약 코드 ▼▼▼ ============================

print("\n===== Aggregating results across all 50 iterations and creating final reports =====")

# --- 1. 최종 성능 요약 (기존과 동일한 방식) ---
bolt_results = np.array(bolt_rmse_lst)
summary = pd.DataFrame({
    'BOLT': np.min(bolt_results, axis=1),
    'Median': all_ens_rmse_lst,
    'INV': inv_rmse_lst,
    'Mean': mean_rmse_lst
})
summary.to_csv(f'inference/{model_name}_{data[d]}_summary_rmse.csv')

final_report_df = summary.describe().T[['mean', 'std']]
final_report_df['mean(std)'] = final_report_df.apply(lambda r: f"{r['mean']:.5f} ({r['std']:.5f})", axis=1)
pd.DataFrame(final_report_df['mean(std)']).T.to_csv(f'{model_name}_{data[d]}_result_ensemble_rmse.csv')
print("\nFinal Performance Summary (Mean over 50 iterations):")
print(final_report_df)


# --- 2. 평균 Breakdown 생성을 위한 데이터 집계 ---
print("\n===== Generating AVERAGE breakdown files over 50 iterations... =====")

# 50번 반복 동안의 모든 예측, 가중치 등을 저장할 리스트
median_preds_50iters, gd_50iters = [], []
bolt_preds_b1, bolt_preds_b3, bolt_preds_b5 = [], [], []
weights_b1, weights_b3, weights_b5 = [], [], []

# samples 리스트를 사용하여 50번의 실험을 동일하게 재구성
for i in range(b):
    nums1, nums2, nums3, nums4, nums5 = samples[i]

    # 기본 예측값(gd)
    fin_pred_mae   = np.median(all[0][nums1], axis=0).flatten()
    fin_pred_mape  = np.median(all[1][nums2], axis=0).flatten()
    fin_pred_mase  = np.median(all[2][nums3], axis=0).flatten()
    fin_pred_mse   = np.median(all[3][nums4], axis=0).flatten()
    fin_pred_smape = np.median(all[4][nums5], axis=0).flatten()
    gd = np.stack([fin_pred_mae, fin_pred_mape, fin_pred_mase, fin_pred_mse, fin_pred_smape], axis=0)
    gd_50iters.append(gd)
    
    # Median 예측값
    score = np.concatenate([all[0][nums1], all[1][nums2], all[2][nums3], all[3][nums4], all[4][nums5]], axis=0)
    median_preds_50iters.append(np.median(score, axis=0).flatten())

    # BOLT 예측 및 가중치 계산을 위한 성능(performance) 값
    mae_val   = np.median(all_val[0][nums1], axis=0).flatten()
    mape_val  = np.median(all_val[1][nums2], axis=0).flatten()
    mase_val  = np.median(all_val[2][nums3], axis=0).flatten()
    mse_val   = np.median(all_val[3][nums4], axis=0).flatten()
    smape_val = np.median(all_val[4][nums5], axis=0).flatten()
    rmse_mae_val   = np.sqrt(mean_squared_error(target_y_val.flatten(), mae_val))
    rmse_mape_val  = np.sqrt(mean_squared_error(target_y_val.flatten(), mape_val))
    rmse_mase_val  = np.sqrt(mean_squared_error(target_y_val.flatten(), mase_val))
    rmse_mse_val   = np.sqrt(mean_squared_error(target_y_val.flatten(), mse_val))
    rmse_smape_val = np.sqrt(mean_squared_error(target_y_val.flatten(), smape_val))
    performance = np.array([rmse_mae_val, rmse_mape_val, rmse_mase_val, rmse_mse_val, rmse_smape_val])

    for target_beta in [1, 3, 5]:
        w = np.exp(-target_beta * performance)
        w /= np.sum(w)
        pred = np.dot(w, gd)
        if target_beta == 1:
            weights_b1.append(w)
            bolt_preds_b1.append(pred)
        elif target_beta == 3:
            weights_b3.append(w)
            bolt_preds_b3.append(pred)
        else: # beta == 5
            weights_b5.append(w)
            bolt_preds_b5.append(pred)

# --- 3. 평균값 계산 ---
avg_median_pred = np.mean(median_preds_50iters, axis=0)
avg_gd = np.mean(gd_50iters, axis=0)
avg_bolt_pred_b1, avg_weights_b1 = np.mean(bolt_preds_b1, axis=0), np.mean(weights_b1, axis=0)
avg_bolt_pred_b3, avg_weights_b3 = np.mean(bolt_preds_b3, axis=0), np.mean(weights_b3, axis=0)
avg_bolt_pred_b5, avg_weights_b5 = np.mean(bolt_preds_b5, axis=0), np.mean(weights_b5, axis=0)

# --- 4. 평균 기반 Breakdown 파일 생성 ---
y_true_flat = test_y.flatten()
num_timesteps = y_true_flat.shape[0]
loss_names = ['mae', 'mape', 'mase', 'mse', 'smape']

# (1) Median Breakdown
avg_median_rmse = np.sqrt(mean_squared_error(y_true_flat, avg_median_pred))
print(f"\nAverage Median RMSE (for breakdown file): {avg_median_rmse:.5f}")
rows = []
for t in range(num_timesteps):
    # ★★★★★ 수정된 부분 1 ★★★★★
    row = {'t': t, 'y_true': y_true_flat[t], 'avg_median_pred': avg_median_pred[t], 'squared_error': (y_true_flat[t] - avg_median_pred[t])**2, 'overall_rmse': avg_median_rmse}
    for loss_idx, name in enumerate(loss_names):
        row[f'avg_base_pred_{name}'] = avg_gd[loss_idx, t]
    rows.append(row)
pd.DataFrame(rows).to_csv(f'inference/{model_name}_{data[d]}_median_breakdown_AVG.csv', index=False)
print(f"INFO: Average Median breakdown saved.")

# (2) BOLT Breakdowns
bolt_scenarios = {1: (avg_bolt_pred_b1, avg_weights_b1), 3: (avg_bolt_pred_b3, avg_weights_b3), 5: (avg_bolt_pred_b5, avg_weights_b5)}
for beta, (avg_pred, avg_w) in bolt_scenarios.items():
    avg_bolt_rmse = np.sqrt(mean_squared_error(y_true_flat, avg_pred))
    print(f"Average BOLT (beta={beta}) RMSE (for breakdown file): {avg_bolt_rmse:.5f}")
    rows = []
    for t in range(num_timesteps):
        # ★★★★★ 수정된 부분 2 ★★★★★
        row = {'t': t, 'beta': beta, 'y_true': y_true_flat[t], 'avg_ensemble_pred': avg_pred[t], 'squared_error': (y_true_flat[t] - avg_pred[t])**2, 'overall_rmse': avg_bolt_rmse}
        for loss_idx, name in enumerate(loss_names):
            row[f'avg_base_pred_{name}'] = avg_gd[loss_idx, t]
            row[f'avg_w_{name}'] = avg_w[loss_idx]
        rows.append(row)
    pd.DataFrame(rows).to_csv(f'inference/{model_name}_{data[d]}_bolt_breakdown_beta_{beta}_AVG.csv', index=False)
    print(f"INFO: Average BOLT (beta={beta}) breakdown saved.")

print("\n===== All processing finished. =====")

--- 파일 검색 디버깅 ---
검색 중인 폴더: resulttf/test/
폴더 안의 모든 파일: ['trTFMLP_coin_mase_pred.csv', 'trTFMLP_coin_mse_pred.csv', 'trTFMLP_coin_mae_pred.csv', 'trTFMLP_coin_smape_pred.csv', 'trTFMLP_coin_mape_pred.csv']
찾고 있는 파일 시작 부분(prefix): 'trTFMLP_coin'
--- 디버깅 끝 ---

INFO (iter 0): Median Ensemble RMSE = 3.64152
INFO: Median breakdown for iter 0 saved.
INFO (iter 0): BOLT (beta=1) RMSE = 3.63317
INFO: BOLT breakdown for beta=1 saved.
INFO (iter 0): BOLT (beta=3) RMSE = 3.63113
INFO: BOLT breakdown for beta=3 saved.
INFO (iter 0): BOLT (beta=5) RMSE = 3.63122
INFO: BOLT breakdown for beta=5 saved.
--------------------

===== Aggregating results across all 50 iterations and creating final reports =====

Final Performance Summary (Mean over 50 iterations):
            mean       std          mean(std)
BOLT    3.631227  0.002300  3.63123 (0.00230)
Median  3.641952  0.002559  3.64195 (0.00256)
INV     3.667214  0.002999  3.66721 (0.00300)
Mean    3.765061  0.043920  3.76506 (0.04392)

===== Generat

In [7]:
data = ['coin']
rec = []
for i in range(len(data)):
    d= pd.read_csv(f'inference/{model_name}_{data[i]}_summary_rmse.csv').iloc[:,1:].describe().T.iloc[:,1:3].round(5)
    d['mean(std)'] = d.apply(
    lambda r: f"{r['mean']:.5f} ({r['std']:.5f})", axis=1)
    
    rec.append(d.loc[:,'mean(std)'])

dfff = pd.DataFrame(rec).T

dfff.columns = data
dfff.to_csv(f'{model_name}_result_ensemble.csv')

In [8]:
dfff = pd.DataFrame(rec).T

dfff.columns = data
dfff.to_csv(f'{model_name}_result_ensemble.csv')

In [9]:
dfff

,coin
BOLT,3.63123 (0.00230)
Median,3.64195 (0.00256)
INV,3.66721 (0.00300)
Mean,3.76506 (0.04392)


In [29]:
a=np.min(np.array(bolt_rmse_lst),axis=0)

print(np.argmin(a)*0.2)

# 251015 정확한 β와, 그 β에서의 평균 weight(부트스트랩 전반)를 같이 출력
idx_global_best = int(np.argmin(a))
beta_grid = np.arange(1, 10.2, 0.2)
beta_global_best = float(np.round(beta_grid[idx_global_best], 3))

# 이 β에서 각 반복이 남긴 weight들을 평균내서 대표 weight를 보여줌
# (weights_by_beta[beta]에는 각 iter의 weight 벡터가 들어있습니       다.)
wlist = np.array(weights_by_beta[beta_global_best])  # shape: (b, 5)
wmean = wlist.mean(axis=0)

print(
    f"[GLOBAL] best β={beta_global_best:.3f} (argmin(a)*0.2={idx_global_best*0.2:.1f}) | "
    f"mean weights(MAE,MAPE,MASE,MSE,SMAPE)="
    f"({wmean[0]:.4f}, {wmean[1]:.4f}, {wmean[2]:.4f}, {wmean[3]:.4f}, {wmean[4]:.4f})"
)

summary= pd.DataFrame({'BOLT' : np.array(bolt_rmse_lst).T[np.argmin(a)],
             'Median' : all_ens_rmse_lst,
             'INV' :inv_rmse_lst,
            'Mean': mean_rmse_lst})
summary.to_csv(f'inference/{model_name}_{data[0]}_summary.csv')
summary.describe()



0.4
[GLOBAL] best β=1.400 (argmin(a)*0.2=0.4) | mean weights(MAE,MAPE,MASE,MSE,SMAPE)=(0.2546, 0.0000, 0.4668, 0.2294, 0.0492)


,BOLT,Median,INV,Mean
count,50.000000,50.000000,50.000000,50.000000
mean,31.430145,33.804334,21.927176,17.229689
std,5.622853,1.849686,1.624610,1.357648
min,11.514178,29.863431,15.670306,14.009819
25%,30.754363,32.603299,21.714360,16.241876
50%,33.082185,33.846544,22.313322,16.911413
75%,34.781210,35.201860,22.662740,18.419089
max,37.076003,37.608830,23.681403,20.515449


In [20]:
# 20251015 ===== [ADD-SAVE-CSV] 요청한 디버깅 산출물 저장 =====
# (1) 손실별 test RMSE 원자료/요약
df_test_rmse = pd.DataFrame(per_loss_test_rmse)
df_test_rmse.index.name = 'bootstrap_iter'
df_test_rmse.to_csv(f'inference/{model_name}_{data[0]}_test_rmse_by_loss.csv', index=True)
df_test_rmse.describe().round(6).to_csv(f'inference/{model_name}_{data[0]}_test_rmse_by_loss_summary.csv')

# (2A) β×부트스트랩별 가중치 원자료 (long)
rows = []
for beta_val, Wlist in weights_by_beta.items():
    for it, w in enumerate(Wlist):
        rows.append({
            'beta': beta_val,
            'iter': it,
            'w_mae':   w[0],
            'w_mape':  w[1],
            'w_mase':  w[2],
            'w_mse':   w[3],
            'w_smape': w[4],
        })
df_weights_raw = pd.DataFrame(rows)
df_weights_raw.to_csv(f'inference/{model_name}_{data[0]}_weights_raw.csv', index=False)

# (2B) β별 가중치 요약(평균/표준편차)
df_weights_summary = (
    df_weights_raw
      .groupby('beta')[['w_mae','w_mape','w_mase','w_mse','w_smape']]
      .agg(['mean','std'])
      .round(6)
)
df_weights_summary.to_csv(f'inference/{model_name}_{data[0]}_weights_summary.csv')

# (2C) 각 부트스트랩에서의 최적 β 및 그때의 가중치/성능
pd.DataFrame(best_beta_rows).to_csv(f'inference/{model_name}_{data[0]}_best_beta_per_iter.csv', index=False)

# === 251015 [ADD] GLOBAL best β에서의 평균 weight와, 샘플 시점 breakdown
idx_global_best = int(np.argmin(a))
beta_grid = np.arange(1, 10.2, 0.2)
beta_global_best = float(np.round(beta_grid[idx_global_best], 3))

# 부트스트랩 전반의 평균 weight
wlist = np.array(weights_by_beta[beta_global_best])  # shape (b, 5)
wmean = wlist.mean(axis=0)
print(f"[GLOBAL] best β={beta_global_best:.3f} | mean weights(MAE,MAPE,MASE,MSE,SMAPE)="
      f"({wmean[0]:.4f}, {wmean[1]:.4f}, {wmean[2]:.4f}, {wmean[3]:.4f}, {wmean[4]:.4f})")

# 원하시면, 전역 최적 β에서의 샘플 breakdown도 CSV로 저장 가능(평균 weight 사용)
# (주의: 평균 weight는 한 iter의 실제 weight와 다르지만, 감 잡기용으로 충분)
sample_k = 0  # 한 시점만 예시
# gd는 iter마다 다르므로, 전역 샘플 breakdown은 의미가 애매할 수 있어 per-iter 쪽(A)로 보시는 걸 권장합니다.

[GLOBAL] best β=1.200 | mean weights(MAE,MAPE,MASE,MSE,SMAPE)=(0.2177, 0.0000, 0.4250, 0.3403, 0.0169)


In [34]:
# ========== 전역 최적 β 1회만 breakdown 저장 ==========
# bolt_rmse_lst: shape (b, n_beta)
arr = np.array(bolt_rmse_lst)
global_i, global_b = np.unravel_index(arr.argmin(), arr.shape)  # iter, beta 인덱스
beta_global_best = float(np.round(beta_grid[global_b], 3))

# 해당 iter에서 썼던 샘플 인덱스 회수
nums1, nums2, nums3, nums4, nums5 = samples[global_i]

# 해당 iter 기준으로 다시 예측/가중치 구성 (동일 방식)
# test 예측 (각 손실별 median)
fin_pred_mae   = np.median(all[0][nums1], axis=0).flatten()
fin_pred_mape  = np.median(all[1][nums2], axis=0).flatten()
fin_pred_mase  = np.median(all[2][nums3], axis=0).flatten()
fin_pred_mse   = np.median(all[3][nums4], axis=0).flatten()
fin_pred_smape = np.median(all[4][nums5], axis=0).flatten()

# val 성능(=가중치 계산에 쓰는 성능)
mae_val   = np.median(all_val[0][nums1], axis=0).flatten()
mape_val  = np.median(all_val[1][nums2], axis=0).flatten()
mase_val  = np.median(all_val[2][nums3], axis=0).flatten()
mse_val   = np.median(all_val[3][nums4], axis=0).flatten()
smape_val = np.median(all_val[4][nums5], axis=0).flatten()

rmse_mae_val   = np.sqrt(mean_squared_error(target_y_val.flatten(), mae_val))
rmse_mape_val  = np.sqrt(mean_squared_error(target_y_val.flatten(), mape_val))
rmse_mase_val  = np.sqrt(mean_squared_error(target_y_val.flatten(), mase_val))
rmse_mse_val   = np.sqrt(mean_squared_error(target_y_val.flatten(), mse_val))
rmse_smape_val = np.sqrt(mean_squared_error(target_y_val.flatten(), smape_val))
performance = np.array([rmse_mae_val, rmse_mape_val, rmse_mase_val, rmse_mse_val, rmse_smape_val])

# 가중치(글로벌 최적 β)
weights = np.exp(-beta_global_best * performance)
weights = weights / weights.sum()

# gd (5, N) & 앙상블
gd = np.vstack([
    fin_pred_mae, fin_pred_mape, fin_pred_mase, fin_pred_mse, fin_pred_smape
])
best_ens_pred = np.dot(weights, gd).flatten()

# 전역 최적 RMSE 계산 및 화면 출력
y_true_flat = test_y.flatten()
global_best_rmse = np.sqrt(mean_squared_error(y_true_flat, best_ens_pred))
print(f"\n[GLOBAL] Best result found at iter={global_i} with beta={beta_global_best:.3f}")
print(f"[GLOBAL] Corresponding RMSE: {global_best_rmse:.6f}")

# breakdown 저장 (이제 딱 1개 파일, 1개 β만 저장)
dbg_rows = []
for k in range(best_ens_pred.shape[0]):
    base_preds_k = gd[:, k]
    recon = float(np.dot(weights, base_preds_k))

    # ✨✨✨ 수정된 부분 ✨✨✨
    # 공통 데이터에 'squared_error'를 추가합니다.
    common_data = {
        'iter': global_i, 'beta': beta_global_best, 't': k,
        'ensemble': best_ens_pred[k], 'reconstructed': recon, 'y_true': y_true_flat[k],
        'squared_error': (y_true_flat[k] - best_ens_pred[k])**2, # 각 시점의 제곱 오차
        'overall_rmse': global_best_rmse
    }

    dbg_rows += [
        {**common_data, 'loss': 'MAE',   'base_pred': base_preds_k[0], 'weight': weights[0], 'weighted': base_preds_k[0]*weights[0]},
        {**common_data, 'loss': 'MAPE',  'base_pred': base_preds_k[1], 'weight': weights[1], 'weighted': base_preds_k[1]*weights[1]},
        {**common_data, 'loss': 'MASE',  'base_pred': base_preds_k[2], 'weight': weights[2], 'weighted': base_preds_k[2]*weights[2]},
        {**common_data, 'loss': 'MSE',   'base_pred': base_preds_k[3], 'weight': weights[3], 'weighted': base_preds_k[3]*weights[3]},
        {**common_data, 'loss': 'SMAPE', 'base_pred': base_preds_k[4], 'weight': weights[4], 'weighted': base_preds_k[4]*weights[4]},
    ]

dbg_path = f'inference/{model_name}_{data[0]}_value_weight_breakdown.csv'
pd.DataFrame(dbg_rows).to_csv(dbg_path, index=False)
print(f"[GLOBAL] breakdown saved → {dbg_path}")


[GLOBAL] Best result found at iter=22 with beta=1.400
[GLOBAL] Corresponding RMSE: 11.514178
[GLOBAL] breakdown saved → inference/trTFMLP_coin_value_weight_breakdown.csv


In [31]:
# ============================ ▼▼▼ 추가할 코드 ▼▼▼ ============================
# 2. 특정 Beta(1,3,5) 값에서의 평균 성능 확인
print("\n--- Performance at Specific Beta Values (RMSE) ---")
target_betas = [1, 3, 5]

# beta_grid를 생성할 때 사용한 np.arange의 부동소수점 오차를 피하기 위해 컬럼명을 반올림합니다.
beta_columns = [round(b, 2) for b in beta_grid]
bolt_results_df = pd.DataFrame(np.array(bolt_rmse_lst), columns=beta_columns)

beta_perf_summary = []
for beta_val in target_betas:
    # .values를 추가하여 Index 객체를 NumPy 배열로 변환
    closest_beta_col = bolt_results_df.columns[np.abs(bolt_results_df.columns.values - beta_val).argmin()]
    
    mean_rmse = bolt_results_df[closest_beta_col].mean()
    std_rmse = bolt_results_df[closest_beta_col].std()
    
    beta_perf_summary.append({
        'target_beta': beta_val,
        'closest_beta': closest_beta_col,
        'mean_rmse': mean_rmse,
        'std_rmse': std_rmse
    })
    print(f"  Beta ≈ {closest_beta_col}: Mean RMSE = {mean_rmse:.5f} (std = {std_rmse:.5f})")

df_beta_summary = pd.DataFrame(beta_perf_summary)
print("\nSpecific Beta Performance Summary:")
print(df_beta_summary)
# ============================ ▲▲▲ 추가할 코드 ▲▲▲ ============================


--- Performance at Specific Beta Values (RMSE) ---
  Beta ≈ 1.0: Mean RMSE = 31.52304 (std = 5.66522)
  Beta ≈ 3.0: Mean RMSE = 31.34087 (std = 5.57777)
  Beta ≈ 5.0: Mean RMSE = 31.31971 (std = 5.56601)

Specific Beta Performance Summary:
   target_beta  closest_beta  mean_rmse  std_rmse
0            1           1.0  31.523039  5.665223
1            3           3.0  31.340869  5.577774
2            5           5.0  31.319713  5.566011


In [32]:
n1  = [1,3,5]
b135 = [] 
for j in range(1):
    a = np.array([np.array(bolt_rmse_lst)[j].mean(axis=0)[np.where(np.arange(1,10.2,0.2) == n1[i])] for i in range(3)])
    b135.append(a)

df135 = pd.DataFrame(np.array(b135).reshape(1,3)).T
df135.columns = data
df135.T.round(3)

IndexError: invalid index to scalar variable.